# 17. Parameter fitting on _A. nidulans_ data

## Numerical experiments - Week 21/2025

_Boyan Mihaylov, MSc Computational Science (UVA/VU)_

Based on the more abundant, time-based data from the _A. niger_ experiments, the model with [1]

## Prerequisite libraries

In [ ]:
using PyPlot
using JLD2
using Statistics
using Revise
using Printf

Revise.includet("./conversions.jl")
Revise.includet("./diffusion.jl")
Revise.includet("./setup.jl")
Revise.includet("./plotting.jl")
Revise.includet("./analysis.jl")
Revise.includet("./datautils.jl")
Revise.includet("./germstats.jl")
using .Conversions
using .Diffusion
using .Setup
using .Plotting
using .Analysis
using .DataUtils
using .GermStats

## 1. Fitting considerations

The _A. nidulans_ dataset does not contain values from which time-dependent evolutions can be reconstructed. Nonetheless, it is noted the measurements are taken at 11 hours of incubation. An alternative study [2] state that the half-saturation time $\tau_g$ for germ tube formation in _A. nidulans_ in a comparable medium is 9.67 hours, the germination rate saturation is 85.46% and the respective design parameter $\nu$ is 6.28. The spore density used in this study is $2.7\times10^5$ spores/mL - at this density, the 11-hour germination rate aligned with the measurements by Herrero-Garcia is estimated to be about 80% [1].

The conflicting findings in the two experiments are rooted in the fact that the 11-hour measurement time in Herrero-Garcia et al [1] is closer to the half-saturation time in the alternative study [2] rather than to a full saturation time. This could suggest that the Dantigny parameters for the data in Herrero-Garcia et al. are different from those in Ijadpanahsaravi et al, likely due to differences in the strain and the cultivation medium used. On the other hand, Herrero-Garcia et al. do not explicitly specify the method used to classify conidia as germinated, which could imply that the percentages also include swollen spores. In any case, it is plausible to assume that the indicated density-dependent germination rates are equilibrium values, i.e. they would not increase significantly under a continued experiment.

With this in consideration, the fitting procedure is required to use **equilibrium probabilities** only. The three models chosen for this are
- inducer-dependent inhibitor threshold and release;
- 2-factor germination with inhibitor-dependent induction threshold;
- independent 2-factor germination with varying permeability.

All models may incorporate a slowly permeating carbon signal, but this assumption becomes irrelevant in the equilibrium limit.

## 2. Equilibrium probabilities

### 2.1. Inducer-dependent inhibitor threshold and release

The time-dependent probability in the first model follows the expression

$$
\begin{equation}
p(\textrm{germination})=\int_0^\infty{\int_0^\infty{\left[1-\Phi{\left(\frac{\beta{(\xi, \pi_{\textrm{eff}})}-\mu_\gamma}{\sigma_\gamma}\right)}\right]\ f_\pi{(\pi_{\textrm{eff}})}\ f_\xi{(\xi)}\ d{\pi_{\textrm{eff}}}}\ d{\xi}}.
\end{equation}
$$

It was previously shown that, in the equilibrium limit, $\beta=\phi$, so the distribution of $\pi_{\textrm{eff}}$, along with its integral, becomes obsolete. Therefore, the new probability takes the form

$$
\begin{equation}
p_{\textrm{eq}}(\textrm{germination})=\int_0^\infty{\left[1-\Phi{\left(\frac{\phi{(\xi)}-\mu_\gamma}{\sigma_\gamma}\right)}\right]\ f_\xi{(\xi)}\ d{\xi}}.
\end{equation}
$$

This is a simple integral which can be solved without resorting to Gauss-Hermite approximation.

### 2.2. Two-factor germination with inhibitor-dependent induction threshold

The time-dependent probability in the second model follows the expression

$$
\begin{equation}
p(\textrm{germination})=\int_0^\infty{\int_0^\infty{\left[1-\Phi{\left(\frac{\beta{(\xi, t)}-\mu_\gamma}{\sigma_\gamma}\right)}\right]\left[\int_0^\infty{\Phi{\left(\frac{s{(\xi, \kappa, t)}-kc_{\textrm{in}}{(\xi, \psi)}-\mu_{\omega_0}}{\sigma_{\omega_0}}\right)}\ f_\psi{(\psi)}\ d{\psi}}\right] f_\kappa{(\kappa)}\ f_\xi{(\xi)}\ d{\kappa}}\ d{\xi}}.
\end{equation}
$$

It was previously shown that, in the equilibrium limit, $\beta=\phi$, hence the equilibrium value of $c_{\textrm{in}}^{\textrm{eq}}=\phi\psi$. Since $c_{\textrm{in}}\to 0$ and $\phi$ is only dependent on $\xi$, the integral over $\psi$ can be omitted. In the long-time limit, the carbon source concentration at the receptor proteins equalises to the ambient concentration in the medium, $c_{\textrm{out}}^{\textrm{cs}}$, so the integral over $\kappa$ can be discarded. Thus, the equilibrium signal strength becomes

$$
\begin{equation}
s_{\textrm{eq}}=\frac{c_{\textrm{out}}^{\textrm{cs}}}{c_{\textrm{out}}^{\textrm{cs}}+K_{\textrm{cs}}}.
\end{equation}
$$

Taking this together, the germination probability simplifies to

$$
\begin{equation}
p_{\textrm{eq}}(\textrm{germination})=\int_0^\infty{\left[1-\Phi{\left(\frac{\phi{(\xi)}-\mu_\gamma}{\sigma_\gamma}\right)}\right]\left[\int_0^\infty{\Phi{\left(\frac{s_{\textrm{eq}}-k\phi{(\xi)}\psi-\mu_{\omega_0}}{\sigma_{\omega_0}}\right)}\ f_\psi{(\psi)}\ d{\psi}}\right]f_\xi{(\xi)}\ d{\xi}}.
\end{equation}
$$

In summary, the residual inhibition still affects both the inhibition-determined germination trigger and the threshold of the inducing signal.

### 2.3. Independent two-factor germination with with varying permeability

The time-dependent probability in the third model follows the expression

$$
\begin{equation}
p(\textrm{germination})=\int_0^\infty{\int_0^\infty{\int_0^\infty{\Phi{\left(\frac{s{(\xi, \kappa, t)}-\mu_\omega}{\sigma_\omega}\right)}\left[1-\Phi{\left(\frac{\beta{(\xi, \alpha, t)}-\mu_\gamma}{\sigma_\gamma}\right)}\right]\ f_\alpha{(\alpha)}\ f_\kappa{(\kappa)}\ f_\xi{(\xi)\ d{\alpha}}}\ d{\kappa}}\ d{\xi}}.
\end{equation}
$$

Since $\beta=\phi$ and $\phi$ is only dependent on $\xi$, the integral over $\alpha$ can be omitted. Similarly, in the long-time limit, the carbon source concentration at the receptor proteins equalises to the ambient concentration in the medium, $c_{\textrm{out}}^{\textrm{cs}}$, making the computations with $t$ and $\kappa$ obsolete. Thus, the germination probability simplifies to

$$
\begin{equation}
p_{\textrm{eq}}(\textrm{germination})=\int_0^\infty{\Phi{\left(\frac{s_{\textrm{eq}}-\mu_\omega}{\sigma_\omega}\right)}\left[1-\Phi{\left(\frac{\phi{(\xi)}-\mu_\gamma}{\sigma_\gamma}\right)}\right]\ f_\xi{(\xi)}\ d{\xi}}.
\end{equation}
$$

It should be noted that this probability does not involve any information about the distribution of the permeation constant or the cell wall porosity, so it is identical with the case of a deterministic permeation constant. Since this is a simple integral, it does not require a Gauss-Hermite approximation.

## References

- [1] Herrero-García, E., Garzia, A., Cordobés, S., Espeso, E.A., & Ugalde, U. (2011). 8-Carbon oxylipins inhibit germination and growth, and stimulate aerial conidiation in Aspergillus nidulans. Fungal biology, 115 4-5, 393-400.
- [2] Ijadpanahsaravi, M., Teertstra, W.R., & Wösten, H.A. (2022). Inter- and intra-species heterogeneity in germination of Aspergillus conidia. Antonie Van Leeuwenhoek, 115, 1151 - 1164.